In [34]:
import cv2
import dlib
import numpy as np
from time import time
from scipy.spatial import distance as dist
from imutils import face_utils
import winsound

def eye_ratio(eyepoint):
    A = dist.euclidean(eyepoint[1],eyepoint[5])
    B = dist.euclidean(eyepoint[2],eyepoint[4])
    C = dist.euclidean(eyepoint[0],eyepoint[3])
    EAR = (A+B) / (2.0*C)
    return EAR

ear = 0.3
sleep = 25
count = 0

(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

cap = cv2.VideoCapture(0)

while True:
    ret, img = cap.read()
    img = cv2.flip(img, 1)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    L = lab[:, :, 0]
    med_L = cv2.medianBlur(L, 99) # median filter
    invert_L = cv2.bitwise_not(med_L) # invert lightness
    composed = cv2.addWeighted(gray, 0.75, invert_L, 0.25, 0)
    img_gray = composed
    
    faces = detector(img_gray)

    for face in faces:
        shape = predictor(img_gray,face)
        shape = face_utils.shape_to_np(shape)
        leftEye = shape[lStart:lEnd]
        rightEye = shape[rStart:rEnd]

        # leftEyeHull = cv2.convexHull(leftEye)
        # rightEyeHull = cv2.convexHull(rightEye)
        # cv2.drawContours(img, [leftEyeHull], -1, (0, 255, 0), 1)
        # cv2.drawContours(img, [rightEyeHull], -1, (0, 255, 0), 1)

        leftEAR = eye_ratio(leftEye)
        rightEAR = eye_ratio(rightEye)
        EAR = leftEAR+rightEAR/2.0
        
        
        if EAR < ear:
            count+=1
            
        if count>sleep:
            winsound.PlaySound("beep-01a.wav", winsound.SND_FILENAME)        # beepsound()
            count=0
            
        cv2.imshow("img",cv2.resize(img, None, fx=0.5, fy=0.5))
    
    if cv2.waitKey(1)==ord("q"):
        break
cap.release() 
cv2.destroyAllWindows()